In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

 
pd.set_option('display.max_columns', None)
# Mostrar mais linhas (padrão é 10)
pd.set_option('display.max_rows', 100)

# Mostrar largura completa da tela
pd.set_option('display.width', None)

# Evita corte de conteúdo dentro das colunas
pd.set_option('display.max_colwidth', None)

In [2]:
# Lista de colunas que você quer carregar
colunas_necessarias = [
    'distribuidora',
    'operacao',
    'duracao_fila',
    'duracao_atendimento',
    'finalizacao',
    'data_hora_intervalo',
    'skill',
    'id_agente_anonimizado',
    'short_call_ate_30',
    'flag_pesquisa',
    'resposta_1',
    'resposta_2',
    'resposta_3',
    'telefone_anonimizado',
    'data_hora_inicio_atendimento',
    'ani_valido'
]

# Ler apenas essas colunas
df = pd.read_csv('C:/Users/Marcelo Almeida/Documents/TRABALHO DE CONCLUSÃO DE CURSO/projeto-tcc/data/raw/aneel_anonimizados.csv', sep=',', encoding='latin1', usecols=colunas_necessarias)
df

C:\Users\Marcelo Almeida\AppData\Local\Temp\ipykernel_21704\3699251360.py:22: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Users/Marcelo Almeida/Documents/TRABALHO DE CONCLUSÃO DE CURSO/projeto-tcc/data/raw/aneel_anonimizados.csv', sep=',', encoding='latin1', usecols=colunas_necessarias)


,distribuidora,operacao,duracao_fila,data_hora_inicio_atendimento,duracao_atendimento,finalizacao,id_agente_anonimizado,data_hora_intervalo,skill,ani_valido,short_call_ate_30,flag_pesquisa,resposta_1,resposta_2,resposta_3,telefone_anonimizado
0,EMPRESA_C,CAC,NaN,2024-08-16 08:44:03,998.0,TRANSFERIDA,69646137742.0,2024-08-16 08:30,GERAL,1,0,1,5.0,1.0,0.0,6.614434e+10
1,EMPRESA_C,CAC,NaN,2024-08-16 08:45:03,389.0,TRANSFERIDA,99574854700.0,2024-08-16 08:30,GERAL,1,0,1,5.0,1.0,0.0,6.914625e+10
2,EMPRESA_D,CAC2,NaN,2024-08-16 08:45:40,303.0,TRANSFERIDA,99906568497894.0,2024-08-16 08:30,FALTA ENERGIA,1,0,1,0.0,0.0,0.0,3.911258e+10
3,EMPRESA_D,CAC,NaN,2024-08-16 08:46:15,399.0,TRANSFERIDA,60991816795.0,2024-08-16 08:30,GERAL,1,0,1,5.0,1.0,0.0,3.911375e+10
4,EMPRESA_B,CAC,NaN,2024-08-16 08:50:07,235.0,TRANSFERIDA,69891149790.0,2024-08-16 08:30,RELIGACAO,1,0,1,0.0,0.0,0.0,9.111689e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16844127,EMPRESA_C,CAC1,0.0,2024-12-03 16:31:56.174,235.0,normal,cec_e.demontier.cantali@cac1.com.br,2024-12-03 16:30:00,GERAL,0,0,0,NaN,NaN,NaN,6.980868e+09
16844128,EMPRESA_C,CAC1,12.0,2024-10-23 13:26:31.817,234.0,normal,cec_ana.carolinas@cac1.com.br,2024-10-23 13:00:00,GERAL,0,0,0,NaN,NaN,NaN,6.811497e+10
16844129,EMPRESA_A,CAC2,0.0,2024-11-22 14:44:01.729,333.0,normal,7452908@cac2.com.br,2024-11-22 14:30:00,FALTA ENERGIA,0,0,0,NaN,NaN,NaN,4.614956e+10
16844130,EMPRESA_A,CAC2,296.0,2024-10-10 22:20:10.705,72.0,normal,7496002@cac2.com.br,2024-10-10 22:00:00,GERAL,0,0,0,NaN,NaN,NaN,4.911704e+10


In [3]:
# Filtrar linhas onde a coluna 'skill' não contém 'TRANSF' (case insensitive)
#df = df[~df["skill"].str.contains("TRANSF", case=False, na=False)]
#df = df[~df["distribuidora"].str.contains("SUPORTE", case=False, na=False)]
#df

In [4]:
# Converter para string corretamente, removendo .0 e mantendo número limpo
df['telefone_anonimizado_novo'] = df['telefone_anonimizado'].astype('Int64').astype(str)
def telefone_simples_valido(numero):
    if pd.isna(numero):
        return 0

    numero = str(numero).strip()

    if len(numero) < 8:
        return 0
    if numero == numero[0] * len(numero):
        return 0

    return 1
df['telefone_valido'] = df['telefone_anonimizado_novo'].apply(telefone_simples_valido)
df

,distribuidora,operacao,duracao_fila,data_hora_inicio_atendimento,duracao_atendimento,finalizacao,id_agente_anonimizado,data_hora_intervalo,skill,ani_valido,short_call_ate_30,flag_pesquisa,resposta_1,resposta_2,resposta_3,telefone_anonimizado,telefone_anonimizado_novo,telefone_valido
0,EMPRESA_C,CAC,NaN,2024-08-16 08:44:03,998.0,TRANSFERIDA,69646137742.0,2024-08-16 08:30,GERAL,1,0,1,5.0,1.0,0.0,6.614434e+10,66144343220,1
1,EMPRESA_C,CAC,NaN,2024-08-16 08:45:03,389.0,TRANSFERIDA,99574854700.0,2024-08-16 08:30,GERAL,1,0,1,5.0,1.0,0.0,6.914625e+10,69146250698,1
2,EMPRESA_D,CAC2,NaN,2024-08-16 08:45:40,303.0,TRANSFERIDA,99906568497894.0,2024-08-16 08:30,FALTA ENERGIA,1,0,1,0.0,0.0,0.0,3.911258e+10,39112584057,1
3,EMPRESA_D,CAC,NaN,2024-08-16 08:46:15,399.0,TRANSFERIDA,60991816795.0,2024-08-16 08:30,GERAL,1,0,1,5.0,1.0,0.0,3.911375e+10,39113749988,1
4,EMPRESA_B,CAC,NaN,2024-08-16 08:50:07,235.0,TRANSFERIDA,69891149790.0,2024-08-16 08:30,RELIGACAO,1,0,1,0.0,0.0,0.0,9.111689e+10,91116889405,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16844127,EMPRESA_C,CAC1,0.0,2024-12-03 16:31:56.174,235.0,normal,cec_e.demontier.cantali@cac1.com.br,2024-12-03 16:30:00,GERAL,0,0,0,NaN,NaN,NaN,6.980868e+09,6980868266,1
16844128,EMPRESA_C,CAC1,12.0,2024-10-23 13:26:31.817,234.0,normal,cec_ana.carolinas@cac1.com.br,2024-10-23 13:00:00,GERAL,0,0,0,NaN,NaN,NaN,6.811497e+10,68114968376,1
16844129,EMPRESA_A,CAC2,0.0,2024-11-22 14:44:01.729,333.0,normal,7452908@cac2.com.br,2024-11-22 14:30:00,FALTA ENERGIA,0,0,0,NaN,NaN,NaN,4.614956e+10,46149560345,1
16844130,EMPRESA_A,CAC2,296.0,2024-10-10 22:20:10.705,72.0,normal,7496002@cac2.com.br,2024-10-10 22:00:00,GERAL,0,0,0,NaN,NaN,NaN,4.911704e+10,49117043531,1


In [5]:
# Filtrar o DataFrame para incluir apenas linhas onde 'ani_valido' é 1 e 'data_hora_inicio_atendimento' não é nulo
df_tratamento = df.copy()
df_tratamento['data_hora_inicio_atendimento'] = df_tratamento['data_hora_inicio_atendimento'].astype(str).str.split('.').str[0]
df_tratamento['data_hora_inicio_atendimento'] = pd.to_datetime(df_tratamento['data_hora_inicio_atendimento'], errors='coerce')
df_tratamento['telefone_valido'] = df_tratamento['telefone_valido'].astype(int)

df_tratamento['linha_valida'] = np.where(
    ((df_tratamento['telefone_valido'] == 1) & (df_tratamento['data_hora_inicio_atendimento'].notnull())),
    1,
    0
)
df_tratamento = df_tratamento[df_tratamento['linha_valida'] == 1]

In [6]:
# Ordenar por cliente e data_hora_inicio_atendimento
df_tratamento = df_tratamento.sort_values(['telefone_anonimizado_novo', 'data_hora_inicio_atendimento'])

df_tratamento['tempo_ate_proxima'] = (
    df_tratamento.groupby('telefone_anonimizado_novo')['data_hora_inicio_atendimento']
    .shift(-1) - df_tratamento['data_hora_inicio_atendimento']
)
# Ver resultado
df_tratamento.head(10) 

,distribuidora,operacao,duracao_fila,data_hora_inicio_atendimento,duracao_atendimento,finalizacao,id_agente_anonimizado,data_hora_intervalo,skill,ani_valido,short_call_ate_30,flag_pesquisa,resposta_1,resposta_2,resposta_3,telefone_anonimizado,telefone_anonimizado_novo,telefone_valido,linha_valida,tempo_ate_proxima
14435013,EMPRESA_D,CAC1,0.0,2025-04-07 17:56:31,287.0,normal,cec_rafaela.catao@cac1.com.br,2025-04-07 17:30:00,GERAL,0,0,0,NaN,NaN,NaN,1.111107e+10,11111071988,1,1,0 days 00:06:54
14401508,EMPRESA_D,CAC1,0.0,2025-04-07 18:03:25,834.0,normal,cec_heleny.costa@cac1.com.br,2025-04-07 18:00:00,GERAL,0,0,0,NaN,NaN,NaN,1.111107e+10,11111071988,1,1,NaT
10163686,EMPRESA_E,CAC2,5.0,2025-02-19 07:43:34,208.0,transferida,7490432@cac2.com.br,2025-02-19 07:30:00,CANCEL ENC,0,0,1,5.0,1.0,NaN,1.111108e+10,11111078104,1,1,0 days 01:16:59
10794347,EMPRESA_E,CAC1,0.0,2025-02-19 09:00:33,449.0,transferida,cec_larissa.rfranca@cac1.com.br,2025-02-19 09:00:00,CANCEL ENC,0,0,1,NaN,NaN,NaN,1.111108e+10,11111078104,1,1,NaT
1533439,EMPRESA_D,CAC,NaN,2024-07-10 11:07:26,70.0,NORMAL,69774959718.0,2024-07-10 11:00,GERAL,1,0,0,NaN,NaN,NaN,1.111109e+10,11111088577,1,1,NaT
10502560,EMPRESA_D,CAC2,0.0,2024-12-09 13:37:00,348.0,transferida,7496085@cac2.com.br,2024-12-09 13:30:00,GERAL,0,0,1,4.0,NaN,NaN,1.111109e+10,11111094291,1,1,0 days 00:20:03
14249444,EMPRESA_D,CAC1,70.0,2024-12-09 13:57:03,189.0,normal,cec_halanice.souto@cac1.com.br,2024-12-09 13:30:00,GERAL,0,0,0,NaN,NaN,NaN,1.111109e+10,11111094291,1,1,NaT
10846989,EMPRESA_D,CAC1,1.0,2025-02-26 08:31:33,220.0,transferida,cec_d.giovana.silva@cac1.com.br,2025-02-26 08:30:00,GERAL,0,0,1,5.0,1.0,NaN,1.111110e+10,11111099233,1,1,NaT
7756436,EMPRESA_C,CAC,NaN,2024-01-05 08:06:36,201.0,TRANSFERIDA,99134058786.0,2024-01-05 08:00,GERAL,1,0,1,0.0,0.0,0.0,1.111127e+10,11111273858,1,1,143 days 11:02:16
2560292,EMPRESA_C,CAC2,NaN,2024-05-27 19:08:52,210.0,TRANSFERIDA,11160879006379.0,2024-05-27 19:00,GERAL,1,0,1,0.0,0.0,0.0,1.111127e+10,11111273858,1,1,176 days 01:00:06


In [7]:
df_tratamento.loc[df_tratamento['telefone_anonimizado_novo'] == '11111094291']

,distribuidora,operacao,duracao_fila,data_hora_inicio_atendimento,duracao_atendimento,finalizacao,id_agente_anonimizado,data_hora_intervalo,skill,ani_valido,short_call_ate_30,flag_pesquisa,resposta_1,resposta_2,resposta_3,telefone_anonimizado,telefone_anonimizado_novo,telefone_valido,linha_valida,tempo_ate_proxima
10502560,EMPRESA_D,CAC2,0.0,2024-12-09 13:37:00,348.0,transferida,7496085@cac2.com.br,2024-12-09 13:30:00,GERAL,0,0,1,4.0,NaN,NaN,1.111109e+10,11111094291,1,1,0 days 00:20:03
14249444,EMPRESA_D,CAC1,70.0,2024-12-09 13:57:03,189.0,normal,cec_halanice.souto@cac1.com.br,2024-12-09 13:30:00,GERAL,0,0,0,NaN,NaN,NaN,1.111109e+10,11111094291,1,1,NaT


In [8]:
df_tratamento['rechamada'] = df_tratamento['tempo_ate_proxima'].apply(lambda x: 0 if pd.isna(x) or x == '' else 1)
df_tratamento['rechamou_30dias'] = df_tratamento['tempo_ate_proxima'].apply(lambda x: 1 if x.total_seconds() < 2592000 else 0)
df_tratamento['rechamou_1h'] = df_tratamento['tempo_ate_proxima'].apply(lambda x: 1 if x.total_seconds() < 3600 else 0)
df_tratamento['rechamou_30min'] = df_tratamento['tempo_ate_proxima'].apply(lambda x: 1 if x.total_seconds() < 1800 else 0)
df_tratamento['rechamou_24h'] = df_tratamento['tempo_ate_proxima'].apply(lambda x: 1 if x.total_seconds() < 86400 else 0)
df_tratamento.head(10)

,distribuidora,operacao,duracao_fila,data_hora_inicio_atendimento,duracao_atendimento,finalizacao,id_agente_anonimizado,data_hora_intervalo,skill,ani_valido,short_call_ate_30,flag_pesquisa,resposta_1,resposta_2,resposta_3,telefone_anonimizado,telefone_anonimizado_novo,telefone_valido,linha_valida,tempo_ate_proxima,rechamada,rechamou_30dias,rechamou_1h,rechamou_30min,rechamou_24h
14435013,EMPRESA_D,CAC1,0.0,2025-04-07 17:56:31,287.0,normal,cec_rafaela.catao@cac1.com.br,2025-04-07 17:30:00,GERAL,0,0,0,NaN,NaN,NaN,1.111107e+10,11111071988,1,1,0 days 00:06:54,1,1,1,1,1
14401508,EMPRESA_D,CAC1,0.0,2025-04-07 18:03:25,834.0,normal,cec_heleny.costa@cac1.com.br,2025-04-07 18:00:00,GERAL,0,0,0,NaN,NaN,NaN,1.111107e+10,11111071988,1,1,NaT,0,0,0,0,0
10163686,EMPRESA_E,CAC2,5.0,2025-02-19 07:43:34,208.0,transferida,7490432@cac2.com.br,2025-02-19 07:30:00,CANCEL ENC,0,0,1,5.0,1.0,NaN,1.111108e+10,11111078104,1,1,0 days 01:16:59,1,1,0,0,1
10794347,EMPRESA_E,CAC1,0.0,2025-02-19 09:00:33,449.0,transferida,cec_larissa.rfranca@cac1.com.br,2025-02-19 09:00:00,CANCEL ENC,0,0,1,NaN,NaN,NaN,1.111108e+10,11111078104,1,1,NaT,0,0,0,0,0
1533439,EMPRESA_D,CAC,NaN,2024-07-10 11:07:26,70.0,NORMAL,69774959718.0,2024-07-10 11:00,GERAL,1,0,0,NaN,NaN,NaN,1.111109e+10,11111088577,1,1,NaT,0,0,0,0,0
10502560,EMPRESA_D,CAC2,0.0,2024-12-09 13:37:00,348.0,transferida,7496085@cac2.com.br,2024-12-09 13:30:00,GERAL,0,0,1,4.0,NaN,NaN,1.111109e+10,11111094291,1,1,0 days 00:20:03,1,1,1,1,1
14249444,EMPRESA_D,CAC1,70.0,2024-12-09 13:57:03,189.0,normal,cec_halanice.souto@cac1.com.br,2024-12-09 13:30:00,GERAL,0,0,0,NaN,NaN,NaN,1.111109e+10,11111094291,1,1,NaT,0,0,0,0,0
10846989,EMPRESA_D,CAC1,1.0,2025-02-26 08:31:33,220.0,transferida,cec_d.giovana.silva@cac1.com.br,2025-02-26 08:30:00,GERAL,0,0,1,5.0,1.0,NaN,1.111110e+10,11111099233,1,1,NaT,0,0,0,0,0
7756436,EMPRESA_C,CAC,NaN,2024-01-05 08:06:36,201.0,TRANSFERIDA,99134058786.0,2024-01-05 08:00,GERAL,1,0,1,0.0,0.0,0.0,1.111127e+10,11111273858,1,1,143 days 11:02:16,1,0,0,0,0
2560292,EMPRESA_C,CAC2,NaN,2024-05-27 19:08:52,210.0,TRANSFERIDA,11160879006379.0,2024-05-27 19:00,GERAL,1,0,1,0.0,0.0,0.0,1.111127e+10,11111273858,1,1,176 days 01:00:06,1,0,0,0,0


In [9]:
#df_trata_filter = df_tratamento[df_tratamento['rechamou_24h'] == 1]
df_trata_filter = df_tratamento

In [10]:
df_trata_filter['short_call_30s'] = (df_trata_filter['duracao_atendimento'] < 30).astype(int)

In [11]:
df_trata_filter['finalizacao'] = df_trata_filter['finalizacao'].str.lower()

In [12]:
df_trata_filter['finalizacao'].value_counts(normalize=True)

finalizacao
transferida           0.637925
normal                0.326660
abandonada em ring    0.025780
erro                  0.007271
abandonada curta      0.001590
cliente               0.000624
consulta              0.000150
Name: proportion, dtype: float64

In [13]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_trata_filter['operacao_codificado'] = le.fit_transform(df_trata_filter['operacao'])
df_trata_filter['skill_codificado'] = le.fit_transform(df_trata_filter['skill'])
df_trata_filter['distribuidora_codificado'] = le.fit_transform(df_trata_filter['distribuidora'])
df_trata_filter['finalizacao_codificado'] = le.fit_transform(df_trata_filter['finalizacao'])
df_trata_filter

,distribuidora,operacao,duracao_fila,data_hora_inicio_atendimento,duracao_atendimento,finalizacao,id_agente_anonimizado,data_hora_intervalo,skill,ani_valido,short_call_ate_30,flag_pesquisa,resposta_1,resposta_2,resposta_3,telefone_anonimizado,telefone_anonimizado_novo,telefone_valido,linha_valida,tempo_ate_proxima,rechamada,rechamou_30dias,rechamou_1h,rechamou_30min,rechamou_24h,short_call_30s,operacao_codificado,skill_codificado,distribuidora_codificado,finalizacao_codificado
14435013,EMPRESA_D,CAC1,0.0,2025-04-07 17:56:31,287.0,normal,cec_rafaela.catao@cac1.com.br,2025-04-07 17:30:00,GERAL,0,0,0,NaN,NaN,NaN,1.111107e+10,11111071988,1,1,0 days 00:06:54,1,1,1,1,1,0,1,24,13,5
14401508,EMPRESA_D,CAC1,0.0,2025-04-07 18:03:25,834.0,normal,cec_heleny.costa@cac1.com.br,2025-04-07 18:00:00,GERAL,0,0,0,NaN,NaN,NaN,1.111107e+10,11111071988,1,1,NaT,0,0,0,0,0,0,1,24,13,5
10163686,EMPRESA_E,CAC2,5.0,2025-02-19 07:43:34,208.0,transferida,7490432@cac2.com.br,2025-02-19 07:30:00,CANCEL ENC,0,0,1,5.0,1.0,NaN,1.111108e+10,11111078104,1,1,0 days 01:16:59,1,1,0,0,1,0,2,2,17,6
10794347,EMPRESA_E,CAC1,0.0,2025-02-19 09:00:33,449.0,transferida,cec_larissa.rfranca@cac1.com.br,2025-02-19 09:00:00,CANCEL ENC,0,0,1,NaN,NaN,NaN,1.111108e+10,11111078104,1,1,NaT,0,0,0,0,0,0,1,2,17,6
1533439,EMPRESA_D,CAC,NaN,2024-07-10 11:07:26,70.0,normal,69774959718.0,2024-07-10 11:00,GERAL,1,0,0,NaN,NaN,NaN,1.111109e+10,11111088577,1,1,NaT,0,0,0,0,0,0,0,24,13,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13080543,EMPRESA_B_OUV,CAC1,0.0,2024-10-21 10:12:23,768.0,normal,cec_e.franklin.paz@cac1.com.br,2024-10-21 10:00:00,OUV SAC,0,0,0,NaN,NaN,NaN,9.989997e+09,9989997000,1,1,9 days 22:05:03,1,1,0,0,0,0,1,37,8,5
10227590,EMPRESA_B_GA,CAC1,0.0,2024-10-31 08:17:26,697.0,normal,cec_e.wirlliane.alves@cac1.com.br,2024-10-31 08:00:00,GA GERAL,0,0,0,NaN,NaN,NaN,9.989997e+09,9989997000,1,1,0 days 00:16:54,1,1,1,1,1,0,1,19,6,5
14004700,EMPRESA_B_OUV,CEC,103.0,2024-10-31 08:34:20,668.0,transferida,denise.santos@cec.com,2024-10-31 08:30:00,OUV RECLAMACOES,0,0,1,4.0,NaN,NaN,9.989997e+09,9989997000,1,1,NaT,0,0,0,0,0,0,3,35,8,6
13999364,EMPRESA_B,CAC1,0.0,2025-04-07 08:14:45,268.0,transferida,cec_sulamita.hellen@cac1.com.br,2025-04-07 08:00:00,LIG NOVA TROCA TIT,0,0,0,NaN,NaN,NaN,9.989998e+09,9989998383,1,1,0 days 00:04:24,1,1,1,1,1,0,1,27,5,6


In [14]:
df_trata_filter['duracao_atendimento'] = df_trata_filter['duracao_atendimento'].fillna(0).astype(int)
df_trata_filter['duracao_fila'] = df_trata_filter['duracao_fila'].fillna(0).astype(int)
df_trata_filter['duracao_fila'] = df_trata_filter['duracao_fila'].fillna(0).astype(int)


In [15]:
df_trata_filter['resposta_1'] = df_trata_filter['resposta_1'].fillna(0).astype(int)
df_trata_filter['resposta_2'] = df_trata_filter['resposta_2'].fillna(0).astype(int)
df_trata_filter['resposta_3'] = df_trata_filter['resposta_3'].fillna(0).astype(int)

In [16]:
# Criar a coluna com a hora cheia
df_trata_filter['hora'] = df_trata_filter['data_hora_inicio_atendimento'].dt.hour
df_trata_filter

,distribuidora,operacao,duracao_fila,data_hora_inicio_atendimento,duracao_atendimento,finalizacao,id_agente_anonimizado,data_hora_intervalo,skill,ani_valido,short_call_ate_30,flag_pesquisa,resposta_1,resposta_2,resposta_3,telefone_anonimizado,telefone_anonimizado_novo,telefone_valido,linha_valida,tempo_ate_proxima,rechamada,rechamou_30dias,rechamou_1h,rechamou_30min,rechamou_24h,short_call_30s,operacao_codificado,skill_codificado,distribuidora_codificado,finalizacao_codificado,hora
14435013,EMPRESA_D,CAC1,0,2025-04-07 17:56:31,287,normal,cec_rafaela.catao@cac1.com.br,2025-04-07 17:30:00,GERAL,0,0,0,0,0,0,1.111107e+10,11111071988,1,1,0 days 00:06:54,1,1,1,1,1,0,1,24,13,5,17
14401508,EMPRESA_D,CAC1,0,2025-04-07 18:03:25,834,normal,cec_heleny.costa@cac1.com.br,2025-04-07 18:00:00,GERAL,0,0,0,0,0,0,1.111107e+10,11111071988,1,1,NaT,0,0,0,0,0,0,1,24,13,5,18
10163686,EMPRESA_E,CAC2,5,2025-02-19 07:43:34,208,transferida,7490432@cac2.com.br,2025-02-19 07:30:00,CANCEL ENC,0,0,1,5,1,0,1.111108e+10,11111078104,1,1,0 days 01:16:59,1,1,0,0,1,0,2,2,17,6,7
10794347,EMPRESA_E,CAC1,0,2025-02-19 09:00:33,449,transferida,cec_larissa.rfranca@cac1.com.br,2025-02-19 09:00:00,CANCEL ENC,0,0,1,0,0,0,1.111108e+10,11111078104,1,1,NaT,0,0,0,0,0,0,1,2,17,6,9
1533439,EMPRESA_D,CAC,0,2024-07-10 11:07:26,70,normal,69774959718.0,2024-07-10 11:00,GERAL,1,0,0,0,0,0,1.111109e+10,11111088577,1,1,NaT,0,0,0,0,0,0,0,24,13,5,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13080543,EMPRESA_B_OUV,CAC1,0,2024-10-21 10:12:23,768,normal,cec_e.franklin.paz@cac1.com.br,2024-10-21 10:00:00,OUV SAC,0,0,0,0,0,0,9.989997e+09,9989997000,1,1,9 days 22:05:03,1,1,0,0,0,0,1,37,8,5,10
10227590,EMPRESA_B_GA,CAC1,0,2024-10-31 08:17:26,697,normal,cec_e.wirlliane.alves@cac1.com.br,2024-10-31 08:00:00,GA GERAL,0,0,0,0,0,0,9.989997e+09,9989997000,1,1,0 days 00:16:54,1,1,1,1,1,0,1,19,6,5,8
14004700,EMPRESA_B_OUV,CEC,103,2024-10-31 08:34:20,668,transferida,denise.santos@cec.com,2024-10-31 08:30:00,OUV RECLAMACOES,0,0,1,4,0,0,9.989997e+09,9989997000,1,1,NaT,0,0,0,0,0,0,3,35,8,6,8
13999364,EMPRESA_B,CAC1,0,2025-04-07 08:14:45,268,transferida,cec_sulamita.hellen@cac1.com.br,2025-04-07 08:00:00,LIG NOVA TROCA TIT,0,0,0,0,0,0,9.989998e+09,9989998383,1,1,0 days 00:04:24,1,1,1,1,1,0,1,27,5,6,8


In [17]:
df_trata_filter.to_csv('C:/Users/Marcelo Almeida/Documents/TRABALHO DE CONCLUSÃO DE CURSO/projeto-tcc/data/processed/aneel_processed.csv', index=False, encoding='utf-8-sig')

In [18]:
contagem = df_trata_filter["data_hora_inicio_atendimento"].dt.to_period("M").value_counts().sort_index()

print(contagem)

data_hora_inicio_atendimento
2024-01    1212787
2024-02    1055259
2024-03    1079674
2024-04     989266
2024-05     932711
2024-06     887604
2024-07     910958
2024-08     928477
2024-09     819894
2024-10    1059731
2024-11     968085
2024-12    1010655
2025-01    1109193
2025-02     959043
2025-03     912836
2025-04     889950
2025-05      67321
Freq: M, Name: count, dtype: int64


In [19]:
display(df_trata_filter)

,distribuidora,operacao,duracao_fila,data_hora_inicio_atendimento,duracao_atendimento,finalizacao,id_agente_anonimizado,data_hora_intervalo,skill,ani_valido,short_call_ate_30,flag_pesquisa,resposta_1,resposta_2,resposta_3,telefone_anonimizado,telefone_anonimizado_novo,telefone_valido,linha_valida,tempo_ate_proxima,rechamada,rechamou_30dias,rechamou_1h,rechamou_30min,rechamou_24h,short_call_30s,operacao_codificado,skill_codificado,distribuidora_codificado,finalizacao_codificado,hora
14435013,EMPRESA_D,CAC1,0,2025-04-07 17:56:31,287,normal,cec_rafaela.catao@cac1.com.br,2025-04-07 17:30:00,GERAL,0,0,0,0,0,0,1.111107e+10,11111071988,1,1,0 days 00:06:54,1,1,1,1,1,0,1,24,13,5,17
14401508,EMPRESA_D,CAC1,0,2025-04-07 18:03:25,834,normal,cec_heleny.costa@cac1.com.br,2025-04-07 18:00:00,GERAL,0,0,0,0,0,0,1.111107e+10,11111071988,1,1,NaT,0,0,0,0,0,0,1,24,13,5,18
10163686,EMPRESA_E,CAC2,5,2025-02-19 07:43:34,208,transferida,7490432@cac2.com.br,2025-02-19 07:30:00,CANCEL ENC,0,0,1,5,1,0,1.111108e+10,11111078104,1,1,0 days 01:16:59,1,1,0,0,1,0,2,2,17,6,7
10794347,EMPRESA_E,CAC1,0,2025-02-19 09:00:33,449,transferida,cec_larissa.rfranca@cac1.com.br,2025-02-19 09:00:00,CANCEL ENC,0,0,1,0,0,0,1.111108e+10,11111078104,1,1,NaT,0,0,0,0,0,0,1,2,17,6,9
1533439,EMPRESA_D,CAC,0,2024-07-10 11:07:26,70,normal,69774959718.0,2024-07-10 11:00,GERAL,1,0,0,0,0,0,1.111109e+10,11111088577,1,1,NaT,0,0,0,0,0,0,0,24,13,5,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13080543,EMPRESA_B_OUV,CAC1,0,2024-10-21 10:12:23,768,normal,cec_e.franklin.paz@cac1.com.br,2024-10-21 10:00:00,OUV SAC,0,0,0,0,0,0,9.989997e+09,9989997000,1,1,9 days 22:05:03,1,1,0,0,0,0,1,37,8,5,10
10227590,EMPRESA_B_GA,CAC1,0,2024-10-31 08:17:26,697,normal,cec_e.wirlliane.alves@cac1.com.br,2024-10-31 08:00:00,GA GERAL,0,0,0,0,0,0,9.989997e+09,9989997000,1,1,0 days 00:16:54,1,1,1,1,1,0,1,19,6,5,8
14004700,EMPRESA_B_OUV,CEC,103,2024-10-31 08:34:20,668,transferida,denise.santos@cec.com,2024-10-31 08:30:00,OUV RECLAMACOES,0,0,1,4,0,0,9.989997e+09,9989997000,1,1,NaT,0,0,0,0,0,0,3,35,8,6,8
13999364,EMPRESA_B,CAC1,0,2025-04-07 08:14:45,268,transferida,cec_sulamita.hellen@cac1.com.br,2025-04-07 08:00:00,LIG NOVA TROCA TIT,0,0,0,0,0,0,9.989998e+09,9989998383,1,1,0 days 00:04:24,1,1,1,1,1,0,1,27,5,6,8


In [20]:
df_processing_base_line = df_trata_filter[['distribuidora_codificado','distribuidora', 'duracao_fila','duracao_atendimento','data_hora_inicio_atendimento','operacao_codificado','operacao','skill_codificado','skill','finalizacao_codificado','finalizacao', 'short_call_30s','hora','flag_pesquisa','resposta_1','resposta_2','resposta_3','rechamou_30min']]

In [21]:
"""

# 2. Ordena os dados cronologicamente
df_processing_base_line = df_processing_base_line.sort_values('data_hora_inicio_atendimento')

# 3. Extrai o mês mais recente da base
ultimo_mes = df_processing_base_line['data_hora_inicio_atendimento'].dt.to_period('M').max()

# 4. Define os 3 últimos meses
ultimos_tres_meses = [ultimo_mes - i for i in range(3)]
ultimos_tres_meses = pd.PeriodIndex(ultimos_tres_meses, freq='M')

# 5. Cria nova coluna apenas com o ano-mês
df_processing_base_line['ano_mes'] = df_processing_base_line['data_hora_inicio_atendimento'].dt.to_period('M')

# 6. Separa os conjuntos
df_holdout_base_line = df_processing_base_line[df_processing_base_line['ano_mes'].isin(ultimos_tres_meses)]        # últimos 3 meses
df_modelagem_base_line = df_processing_base_line[~df_processing_base_line['ano_mes'].isin(ultimos_tres_meses)]      # restante para treino/validação
"""

"\n\n# 2. Ordena os dados cronologicamente\ndf_processing_base_line = df_processing_base_line.sort_values('data_hora_inicio_atendimento')\n\n# 3. Extrai o mês mais recente da base\nultimo_mes = df_processing_base_line['data_hora_inicio_atendimento'].dt.to_period('M').max()\n\n# 4. Define os 3 últimos meses\nultimos_tres_meses = [ultimo_mes - i for i in range(3)]\nultimos_tres_meses = pd.PeriodIndex(ultimos_tres_meses, freq='M')\n\n# 5. Cria nova coluna apenas com o ano-mês\ndf_processing_base_line['ano_mes'] = df_processing_base_line['data_hora_inicio_atendimento'].dt.to_period('M')\n\n# 6. Separa os conjuntos\ndf_holdout_base_line = df_processing_base_line[df_processing_base_line['ano_mes'].isin(ultimos_tres_meses)]        # últimos 3 meses\ndf_modelagem_base_line = df_processing_base_line[~df_processing_base_line['ano_mes'].isin(ultimos_tres_meses)]      # restante para treino/validação\n"

In [22]:
# 2. Ordena os dados cronologicamente
df_processing_base_line = df_processing_base_line.sort_values('data_hora_inicio_atendimento')

# 3. Extrai o mês mais recente da base
ultimo_mes = df_processing_base_line['data_hora_inicio_atendimento'].dt.to_period('M').max()

# 4. Define os 3 últimos meses
ultimos_tres_meses = [ultimo_mes - i for i in range(3)]
ultimos_tres_meses = pd.PeriodIndex(ultimos_tres_meses, freq='M')

# 5. Cria nova coluna apenas com o ano-mês
df_processing_base_line['ano_mes'] = df_processing_base_line['data_hora_inicio_atendimento'].dt.to_period('M')

# 6. Separa os conjuntos
df_holdout_base_line = df_processing_base_line[df_processing_base_line['ano_mes'].isin(ultimos_tres_meses)]        # últimos 3 meses

primeiro_mes = df_holdout_base_line['data_hora_inicio_atendimento'].dt.to_period('M').min() -1

# 4. Define os 3 últimos meses
ultimos_seis_meses = [primeiro_mes - i for i in range(5)]
ultimos_seis_meses = pd.PeriodIndex(ultimos_seis_meses, freq='M') 

df_modelagem_base_line = df_processing_base_line[df_processing_base_line['ano_mes'].isin(ultimos_seis_meses)] 

In [23]:
df_processing_base_line['rechamou_30min'].value_counts(normalize=True)

rechamou_30min
0    0.841603
1    0.158397
Name: proportion, dtype: float64

In [24]:
print("Últimos 3 meses:", ultimos_tres_meses)
print("Últimos 6 meses:", ultimos_seis_meses)
print("Tamanho do holdout:", df_holdout_base_line.shape)
print("Tamanho do restante:", df_modelagem_base_line.shape)

Últimos 3 meses: PeriodIndex(['2025-05', '2025-04', '2025-03'], dtype='period[M]')
Últimos 6 meses: PeriodIndex(['2025-02', '2025-01', '2024-12', '2024-11', '2024-10'], dtype='period[M]')
Tamanho do holdout: (1870107, 19)
Tamanho do restante: (5106707, 19)


In [25]:
#salvar o DataFrame filtrado em um arquivo CSV
df_holdout_base_line.to_csv('C:/Users/Marcelo Almeida/Documents/TRABALHO DE CONCLUSÃO DE CURSO/projeto-tcc/data/processed/base_line/aneel_processed_holdout_baseline.csv', index=False, encoding='utf-8-sig')
df_modelagem_base_line.to_csv('C:/Users/Marcelo Almeida/Documents/TRABALHO DE CONCLUSÃO DE CURSO/projeto-tcc/data/processed/base_line/aneel_processed_model_baseline.csv', index=False, encoding='utf-8-sig')

In [26]:
#df_processing.to_csv('C:/Users/Marcelo Almeida/Documents/TRABALHO DE CONCLUSÃO DE CURSO/projeto-tcc/data/processed/aneel_processed_diff.csv', index=False, encoding='utf-8-sig')